In [1]:
import pandas as pd
import numpy as np

from Code.UtilityFunctions.get_data_path import get_path
from Code.UtilityFunctions.run_query import run_query

In [2]:
business = pd.read_json(get_path("yelp_academic_dataset_business.json"), lines=True)
business['categories'] = business['categories'].str.split(', ', expand=False)

In [3]:
#users = pd.read_json(get_path("yelp_academic_dataset_user.json"), lines=True)
#reviews = pd.read_json(get_path("yelp_academic_dataset_review.json"), lines=

checkins = pd.read_json(get_path("yelp_academic_dataset_checkin.json"), lines=True)
checkins['date'] = checkins['date'].str.split(', ', expand=False)

tips = pd.read_json(get_path("yelp_academic_dataset_tip.json"), lines=True)

In [ ]:
business

In [ ]:
checkins

In [ ]:
tips

 How many different types of businesses are defined in Yelp?

In [4]:
sparql_query = """

SELECT COUNT(DISTINCT(?category))
WHERE {
    ?business schema:category ?category .
}

"""

run_query(query=sparql_query, as_dataframe=True)

,callret-0.value
0,1311


In [3]:
unique_categories = {category for sublist in business['categories'] if sublist for category in sublist}
len(unique_categories)

1311

How many businesses of type "Restaurants" exist?

In [6]:
sparql_query = """

SELECT COUNT(?business)
WHERE {
    ?business schema:category yelp_category:Restaurants .
}

"""

run_query(query=sparql_query, as_dataframe=True)

,callret-0.value
0,52268


In [10]:
# TO DO Python
# def str_split(string):
#     if isinstance(string, str):
#         return string.split(", ")
#     else:
#         return string
#
#
# business["categories"] = business["categories"].apply(str_split)
#
# category_biz_count = {}
# for i in range(len(business)):
#     if business["categories"][i]:
#         for category in business["categories"][i]:
#             category_biz_count[category] = category_biz_count.get(category, 0) + 1
#
# category_biz_count["Restaurants"]

counter = 0
for row in business['categories']:
    if row is not None and "Restaurants" in row:
        counter += 1
counter

52268

How many businesses of type Restaurants have been reviwed?

In [7]:
sparql_query = """

SELECT COUNT(DISTINCT(?business))
WHERE {
    ?business schema:category yelp_category:Restaurants .
    ?review schema:about ?business .
}

"""

run_query(query=sparql_query, as_dataframe=True)

,callret-0.value
0,0


In [ ]:
business_and_tips = tips.merge(business[['business_id', 'categories']])

business_and_tips

In [ ]:
indexes = business_and_tips["Restaurants" in business_and_tips['categories']].index

In [ ]:
for row in business_and_tips.itertuples():
    idx = row.Index
    categories = row.categories
    if 'Restaurants' not in categories:
        business_and_tips.drop(index=idx, inplace=True)

In [ ]:
business_and_tips

How many businesses have been reviewed?

In [8]:
sparql_query = """

SELECT COUNT(?business)
WHERE {
    ?review schema:about ?business .
    ?business rdfs.Class schema:LocalBusiness .
}

"""

run_query(query=sparql_query, as_dataframe=True)

,callret-0.value
0,0


In [ ]:
tips['business_id'].unique.count()

How many businesses have, on average, a rating of 4.5?

In [12]:
sparql_query = """
SELECT COUNT(DISTINCT(?business))
WHERE {
    ?business schema:aggregateRating ?rating .
    FILTER (?rating = 4.5) .
}

"""

run_query(query=sparql_query, as_dataframe=True)

,callret-0.value
0,27174


In [47]:
business.groupby('stars')['stars'].count()

stars
1.0     1986
1.5     4932
2.0     9527
2.5    14316
3.0    18453
3.5    26519
4.0    31125
4.5    27181
5.0    16307
Name: stars, dtype: int64

What is the average rating across businesses?

In [11]:
sparql_query = """
SELECT AVG(?rating) as ?averagerating
WHERE {
    ?business schema:aggregateRating ?rating .
}

"""

run_query(query=sparql_query, as_dataframe=True)

,averagerating.type,averagerating.datatype,averagerating.value
0,typed-literal,http://www.w3.org/2001/XMLSchema#decimal,3.59672355766033


In [12]:
business['stars'].mean()

3.5967235576603303

How many businesses have been reviewed in New York, NY?

In [15]:
sparql_query = """
SELECT ?state ?city COUNT(?business) AS ?count_business
WHERE {
    ?business schema:category ?category .
    ?business schema:addressRegion ?state .
    ?business schema:location ?city .
    FILTER(?city = "New York")
}
GROUP BY ?state ?city
ORDER BY DESC(?count_business)
LIMIT 10

"""

run_query(query=sparql_query, as_dataframe=True)

Empty resultset


""


In [15]:
business.groupby(["state", "city"])["business_id"].count().sort_values(ascending=False)[0:10]

state  city         
PA     Philadelphia     14567
AZ     Tucson            9249
FL     Tampa             9048
IN     Indianapolis      7540
TN     Nashville         6968
LA     New Orleans       6208
NV     Reno              5932
AB     Edmonton          5054
MO     Saint Louis       4827
CA     Santa Barbara     3829
Name: business_id, dtype: int64

Which business is the most visited on day X?

In [32]:
sparql_query = """
SELECT ?business ?year ?month ?day COUNT(?visit)
WHERE {
    ?business schema:checkinTime ?visit .
    BIND (day(?visit)  as ?day)
    BIND (month(?visit) as ?month)
    BIND (year(?visit) as ?year)
}
GROUP BY ?business ?year ?month ?day
ORDER BY DESC(COUNT(?visit))
LIMIT 5

"""

run_query(query=sparql_query, as_dataframe=True)

,business.value,year.value,month.value,day.value,callret-4.value
0,https://example.org/business_id/CySqUcNz8oPiQTu4EXTnig,2016,6,25,457
1,https://example.org/business_id/g50ImmCX3WY3koEDIzoKxg,2015,8,30,285
2,https://example.org/business_id/qfWWx0dVo1UuAhRfh03Dyw,2016,8,28,268
3,https://example.org/business_id/g50ImmCX3WY3koEDIzoKxg,2016,8,28,263
4,https://example.org/business_id/FBBeJO50xZiNIo3oFhAFRA,2017,7,29,251


Which are the top 10 most visisted businesses?

In [2]:
sparql_query = """
SELECT DISTINCT ?business ?url COUNT(?visit)
WHERE {
    ?business schema:checkinTime ?visit .
    ?business schema:url ?url
    
}
GROUP BY ?business ?url
ORDER BY DESC(COUNT(?visit))
LIMIT 10

"""

run_query(query=sparql_query, as_dataframe=True)

,business.value,url.value,callret-2.value
0,https://example.org/business_id/-QI8Qi8XWH3D8y8ethnajA,https://www.yelp.com/biz/-QI8Qi8XWH3D8y8ethnajA,52129
1,https://example.org/business_id/FEXhWNCMkv22qG04E83Qjg,https://www.yelp.com/biz/FEXhWNCMkv22qG04E83Qjg,40092
2,https://example.org/business_id/Eb1XmmLWyt_way5NNZ7-Pw,https://www.yelp.com/biz/Eb1XmmLWyt_way5NNZ7-Pw,37553
3,https://example.org/business_id/c_4c5rJECZSfNgFj7frwHQ,https://www.yelp.com/biz/c_4c5rJECZSfNgFj7frwHQ,37511
4,https://example.org/business_id/4i4kmYm9wgSNyF1b6gKphg,https://www.yelp.com/biz/4i4kmYm9wgSNyF1b6gKphg,31163
5,https://example.org/business_id/8O35ji_yOMVJmZ6bl96yhQ,https://www.yelp.com/biz/8O35ji_yOMVJmZ6bl96yhQ,29599
6,https://example.org/business_id/VQcCL9PiNL_wkGf-uF3fjg,https://www.yelp.com/biz/VQcCL9PiNL_wkGf-uF3fjg,28917
7,https://example.org/business_id/ac1AeYqs8Z4_e2X5M3if2A,https://www.yelp.com/biz/ac1AeYqs8Z4_e2X5M3if2A,21527
8,https://example.org/business_id/QTbahs-GVuWYL5yfdjH34A,https://www.yelp.com/biz/QTbahs-GVuWYL5yfdjH34A,21478
9,https://example.org/business_id/ytynqOUb3hjKeJfRj5Tshw,https://www.yelp.com/biz/ytynqOUb3hjKeJfRj5Tshw,18604


In [36]:
sparql_query = """
SELECT ?business COUNT(?visit)
WHERE {
    ?business schema:checkinTime ?visit .
}
GROUP BY ?business 
ORDER BY DESC(COUNT(?visit))
LIMIT 10

"""

run_query(query=sparql_query, as_dataframe=True)

,business.value,callret-1.value
0,https://example.org/business_id/-QI8Qi8XWH3D8y8ethnajA,52129
1,https://example.org/business_id/FEXhWNCMkv22qG04E83Qjg,40092
2,https://example.org/business_id/Eb1XmmLWyt_way5NNZ7-Pw,37553
3,https://example.org/business_id/c_4c5rJECZSfNgFj7frwHQ,37511
4,https://example.org/business_id/4i4kmYm9wgSNyF1b6gKphg,31163
5,https://example.org/business_id/8O35ji_yOMVJmZ6bl96yhQ,29599
6,https://example.org/business_id/VQcCL9PiNL_wkGf-uF3fjg,28917
7,https://example.org/business_id/ac1AeYqs8Z4_e2X5M3if2A,21527
8,https://example.org/business_id/QTbahs-GVuWYL5yfdjH34A,21478
9,https://example.org/business_id/ytynqOUb3hjKeJfRj5Tshw,18604


In [53]:
checkins

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"[2020-03-13 21:10:56, 2020-06-02 22:18:06, 202..."
1,--0iUa4sNDFiZFrAdIWhZQ,"[2010-09-13 21:43:09, 2011-05-04 23:08:15, 201..."
2,--30_8IhuyMHbSOcNWd6DQ,"[2013-06-14 23:29:17, 2014-08-13 23:20:22]"
3,--7PUidqRWpRSpXebiyxTg,"[2011-02-15 17:12:00, 2011-07-28 02:46:10, 201..."
4,--7jw19RH9JKXgFohspgQw,"[2014-04-21 20:42:11, 2014-04-28 21:04:46, 201..."
...,...,...
131925,zznJox6-nmXlGYNWgTDwQQ,"[2013-03-23 16:22:47, 2013-04-07 02:03:12, 201..."
131926,zznZqH9CiAznbkV6fXyHWA,[2021-06-12 01:16:12]
131927,zzu6_r3DxBJuXcjnOYVdTw,"[2011-05-24 01:35:13, 2012-01-01 23:44:33, 201..."
131928,zzw66H6hVjXQEt0Js3Mo4A,"[2016-12-03 23:33:26, 2018-12-02 19:08:45]"


In [55]:
checkins = checkins.explode('date')

In [59]:
checkins.groupby(['business_id']).size()

business_id
---kPU91CF4Lq2-WlRu9Lw    11
--0iUa4sNDFiZFrAdIWhZQ    10
--30_8IhuyMHbSOcNWd6DQ     2
--7PUidqRWpRSpXebiyxTg    10
--7jw19RH9JKXgFohspgQw    26
                          ..
zznJox6-nmXlGYNWgTDwQQ    67
zznZqH9CiAznbkV6fXyHWA     1
zzu6_r3DxBJuXcjnOYVdTw    23
zzw66H6hVjXQEt0Js3Mo4A     2
zzyx5x0Z7xXWWvWnZFuxlQ     1
Length: 131930, dtype: int64

How many of the Yelp categories are also mapped to schema.org?

In [ ]:
sparql_query = """
SELECT COUNT(?category)
WHERE {
    ?category rdfs.Class example:yelpCategory.
    ?category skos:exactMatch ?schemacategory .
    ?category skos:narrowMatch ?schemacategory .
    ?schemacategory rdfs.Class example:SchemaCategory .

}
"""

run_query(query=sparql_query, as_dataframe=True)

In [63]:
mappings = pd.read_csv(get_path("class_mappings.csv"))
len(mappings)

384

How many people have written a review or tip on Yelp? TO DO: review author user, user author tip giver ikke mening

In [37]:
sparql_query = """
SELECT COUNT(?users)
WHERE {
    ?user rdfs:Class schema:Person .
    ?review schema:author ?user .


}
"""

run_query(query=sparql_query, as_dataframe=True)

,callret-0.type,callret-0.datatype,callret-0.value
0,typed-literal,http://www.w3.org/2001/XMLSchema#integer,0


In [ ]:
tips['user_id'].unique.count()

How many users have 10 friends?

In [ ]:
sparql_query = """
SELECT ?user (COUNT(?friend) as ?numberOfFriends)
WHERE {
    ?user rdfs:Class schema:Person .
    ?user schema:knows ?friend .
}
HAVING { ?numberOfFriends = 10 }
GROUP BY ?user
"""

run_query(query=sparql_query, as_dataframe=True)

How many friends does a user have on average?

In [ ]:
sparql_query = """
SELECT (AVG(?friend) as ?averageFriends)
WHERE {
    ?user rdfs:Class schema:Person .
    ?user schema:knows ?friend .
}
"""

run_query(query=sparql_query, as_dataframe=True)

How many users have authored 10 reviews or tips?

In [ ]:
sparql_query = """
SELECT ?user (COUNT(?reviews) as ?numberOfReviews)
WHERE {
    ?user rdfs:Class schema:Person .
    ?user schema:knows ?friend .
}
HAVING { ?numberOfFriends = 10 }
GROUP BY ?user
"""

run_query(query=sparql_query, as_dataframe=True)

In [64]:
tips

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0
...,...,...,...,...,...
908910,eYodOTF8pkqKPzHkcxZs-Q,3lHTewuKFt5IImbXJoFeDQ,Disappointed in one of your managers.,2021-09-11 19:18:57,0
908911,1uxtQAuJ2T5Xwa_wp7kUnA,OaGf0Dp56ARhQwIDT90w_g,Great food and service.,2021-10-30 11:54:36,0
908912,v48Spe6WEpqehsF2xQADpg,hYnMeAO77RGyTtIzUSKYzQ,Love their Cubans!!,2021-11-05 13:18:56,0
908913,ckqKGM2hl7I9Chp5IpAhkw,s2eyoTuJrcP7I_XyjdhUHQ,Great pizza great price,2021-11-20 16:11:44,0


In [ ]:
top_tippers = tips.groupby('user_id')['user_id'].count()

How many reviews or tips does a user make on average every month?

In [ ]:
# TO DO